# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 10:24AM,237847,19,60015310,"GUSA Granules USA, Inc.",Released
1,May 23 2022 10:13AM,237841,10,0085695147,ISDIN Corporation,Released
2,May 23 2022 10:13AM,237841,10,0085695149,ISDIN Corporation,Released
3,May 23 2022 10:13AM,237841,10,0085695151,ISDIN Corporation,Released
4,May 23 2022 10:13AM,237841,10,0085695154,ISDIN Corporation,Released
5,May 23 2022 10:13AM,237841,10,0085695158,ISDIN Corporation,Released
6,May 23 2022 10:13AM,237841,10,0085695160,ISDIN Corporation,Released
7,May 23 2022 10:13AM,237841,10,0085695163,ISDIN Corporation,Released
8,May 23 2022 10:13AM,237841,10,0085695169,ISDIN Corporation,Released
9,May 23 2022 10:13AM,237841,10,0085695173,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,237839,Released,3
39,237840,Released,1
40,237841,Released,58
41,237843,Released,1
42,237847,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237839,NaN,NaN,3.0
237840,NaN,NaN,1.0
237841,NaN,NaN,58.0
237843,NaN,NaN,1.0
237847,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237781,0.0,0.0,51.0
237787,0.0,0.0,1.0
237789,1.0,6.0,0.0
237792,0.0,0.0,18.0
237794,0.0,0.0,26.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237781,0,0,51
237787,0,0,1
237789,1,6,0
237792,0,0,18
237794,0,0,26


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237781,0,0,51
1,237787,0,0,1
2,237789,1,6,0
3,237792,0,0,18
4,237794,0,0,26


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237781,,,51
1,237787,,,1
2,237789,1,6,
3,237792,,,18
4,237794,,,26


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc."
1,May 23 2022 10:13AM,237841,10,ISDIN Corporation
59,May 23 2022 10:09AM,237843,10,ISDIN Corporation
60,May 23 2022 10:08AM,237836,10,ISDIN Corporation
111,May 23 2022 9:59AM,237840,12,"Organimed, LLC"
112,May 23 2022 9:58AM,237839,10,ISDIN Corporation
115,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC"
116,May 23 2022 9:53AM,237830,10,ISDIN Corporation
179,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC"
180,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc.",,,1
1,May 23 2022 10:13AM,237841,10,ISDIN Corporation,,,58
2,May 23 2022 10:09AM,237843,10,ISDIN Corporation,,,1
3,May 23 2022 10:08AM,237836,10,ISDIN Corporation,,,51
4,May 23 2022 9:59AM,237840,12,"Organimed, LLC",,,1
5,May 23 2022 9:58AM,237839,10,ISDIN Corporation,,,3
6,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC",,,1
7,May 23 2022 9:53AM,237830,10,ISDIN Corporation,,,63
8,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC",,,1
9,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc.",1,,
1,May 23 2022 10:13AM,237841,10,ISDIN Corporation,58,,
2,May 23 2022 10:09AM,237843,10,ISDIN Corporation,1,,
3,May 23 2022 10:08AM,237836,10,ISDIN Corporation,51,,
4,May 23 2022 9:59AM,237840,12,"Organimed, LLC",1,,
5,May 23 2022 9:58AM,237839,10,ISDIN Corporation,3,,
6,May 23 2022 9:56AM,237837,16,"Emersa Waterbox, LLC",1,,
7,May 23 2022 9:53AM,237830,10,ISDIN Corporation,63,,
8,May 23 2022 9:48AM,237832,19,"Bi-Coastal Pharma International, LLC",1,,
9,May 23 2022 9:46AM,237829,18,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc.",1,,
1,May 23 2022 10:13AM,237841,10,ISDIN Corporation,58,,
2,May 23 2022 10:09AM,237843,10,ISDIN Corporation,1,,
3,May 23 2022 10:08AM,237836,10,ISDIN Corporation,51,,
4,May 23 2022 9:59AM,237840,12,"Organimed, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
1,May 23 2022 10:13AM,237841,10,ISDIN Corporation,58.0,NaN,NaN
2,May 23 2022 10:09AM,237843,10,ISDIN Corporation,1.0,NaN,NaN
3,May 23 2022 10:08AM,237836,10,ISDIN Corporation,51.0,NaN,NaN
4,May 23 2022 9:59AM,237840,12,"Organimed, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 10:24AM,237847,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
1,May 23 2022 10:13AM,237841,10,ISDIN Corporation,58.0,0.0,0.0
2,May 23 2022 10:09AM,237843,10,ISDIN Corporation,1.0,0.0,0.0
3,May 23 2022 10:08AM,237836,10,ISDIN Corporation,51.0,0.0,0.0
4,May 23 2022 9:59AM,237840,12,"Organimed, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4280760,423.0,19.0,0.0
12,237840,1.0,0.0,0.0
15,1426797,80.0,0.0,0.0
16,237837,1.0,0.0,0.0
18,1664668,16.0,1.0,0.0
19,713468,2.0,6.0,1.0
22,237787,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4280760,423.0,19.0,0.0
1,12,237840,1.0,0.0,0.0
2,15,1426797,80.0,0.0,0.0
3,16,237837,1.0,0.0,0.0
4,18,1664668,16.0,1.0,0.0
5,19,713468,2.0,6.0,1.0
6,22,237787,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,423.0,19.0,0.0
1,12,1.0,0.0,0.0
2,15,80.0,0.0,0.0
3,16,1.0,0.0,0.0
4,18,16.0,1.0,0.0
5,19,2.0,6.0,1.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,423.0
1,12,Released,1.0
2,15,Released,80.0
3,16,Released,1.0
4,18,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Executing,19.0,0.0,0.0,0.0,1.0,6.0,0.0
Released,423.0,1.0,80.0,1.0,16.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,22
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,19.0,0.0,0.0,0.0,1.0,6.0,0.0
2,Released,423.0,1.0,80.0,1.0,16.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,22
0,Completed,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,19.0,0.0,0.0,0.0,1.0,6.0,0.0
2,Released,423.0,1.0,80.0,1.0,16.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()